In [1]:
%load_ext autoreload
%autoreload 2

import copy
import os
import torch
import numpy as np
import pythreejs as pjs
import warnings
warnings.filterwarnings('ignore')

from matplotlib.cm import get_cmap
from tqdm.notebook import tqdm
from PIL import Image
from IPython.core.display import display

os.chdir('..')
np.set_printoptions(precision=4)
!pwd

/home/lcw/workspace/panosupervised


In [2]:
from vidar.arch.models.depth.PanoSupervisedModel import PanoSupervisedModel
from vidar.arch.networks.depth.MultiCamDepthNet import MultiCamDepthNet

In [3]:
weight_path = '/data/disk/logs/checkpoints/soft-tree-60/models/017.ckpt'
# model = Depth()
state_dict = torch.load(weight_path)

In [4]:
state_dict.keys()

dict_keys(['config', 'epoch', 'state_dict'])

In [5]:
model = state_dict['state_dict']

In [6]:
config = state_dict['config']

In [7]:
config

Config(arch=Config(losses=Config(smoothness=Config(file='SmoothnessLoss', gamma=0.5, normalize=True, weight=0.0001), supervised=Config(automask_loss=True, file='SupervisedDepthLoss', flow_downsampling=False, gamma=1.0, mask_sparse=True, method='huber', upsample_depth=False)), model=Config(file='depth/PanoSupervisedModel'), networks=Config(depth=Config(decoder=Config(activation='sigmoid', depth_focal=1.0, depth_hypothesis=[2, 3, 10, 30, 60, 90], downsample=False, file='decoders/PanoDepthDecoder', fusion_type='MultiDepthSweepFusion', input_cameras=['camera_01', 'camera_05', 'camera_06', 'camera_07', 'camera_08', 'camera_09'], num_ch_out=1, out_shape=[128, 1024], positional_encoding=16, ref_shape=[256, 2048], scale_and_shapes={'camera_01': [(2, (64, 192, 320), 4, (64, 64, 512)), (4, (64, 96, 160), 8, (64, 32, 256)), (8, (128, 48, 80), 16, (128, 16, 128)), (16, (256, 24, 40), 32, (256, 8, 64)), (32, (512, 12, 20), 64, (512, 4, 32))], 'camera_05': [(2, (64, 192, 320), 4, (64, 64, 512)), (4,

In [8]:
for k, v in state_dict['state_dict'].items():
    if 'encoder' in k:
        print(k)

networks.depth.networks.encoders.camera_01.encoder.conv1.weight
networks.depth.networks.encoders.camera_01.encoder.bn1.weight
networks.depth.networks.encoders.camera_01.encoder.bn1.bias
networks.depth.networks.encoders.camera_01.encoder.bn1.running_mean
networks.depth.networks.encoders.camera_01.encoder.bn1.running_var
networks.depth.networks.encoders.camera_01.encoder.bn1.num_batches_tracked
networks.depth.networks.encoders.camera_01.encoder.layer1.0.conv1.weight
networks.depth.networks.encoders.camera_01.encoder.layer1.0.bn1.weight
networks.depth.networks.encoders.camera_01.encoder.layer1.0.bn1.bias
networks.depth.networks.encoders.camera_01.encoder.layer1.0.bn1.running_mean
networks.depth.networks.encoders.camera_01.encoder.layer1.0.bn1.running_var
networks.depth.networks.encoders.camera_01.encoder.layer1.0.bn1.num_batches_tracked
networks.depth.networks.encoders.camera_01.encoder.layer1.0.conv2.weight
networks.depth.networks.encoders.camera_01.encoder.layer1.0.bn2.weight
networks.d

In [9]:
config

Config(arch=Config(losses=Config(smoothness=Config(file='SmoothnessLoss', gamma=0.5, normalize=True, weight=0.0001), supervised=Config(automask_loss=True, file='SupervisedDepthLoss', flow_downsampling=False, gamma=1.0, mask_sparse=True, method='huber', upsample_depth=False)), model=Config(file='depth/PanoSupervisedModel'), networks=Config(depth=Config(decoder=Config(activation='sigmoid', depth_focal=1.0, depth_hypothesis=[2, 3, 10, 30, 60, 90], downsample=False, file='decoders/PanoDepthDecoder', fusion_type='MultiDepthSweepFusion', input_cameras=['camera_01', 'camera_05', 'camera_06', 'camera_07', 'camera_08', 'camera_09'], num_ch_out=1, out_shape=[128, 1024], positional_encoding=16, ref_shape=[256, 2048], scale_and_shapes={'camera_01': [(2, (64, 192, 320), 4, (64, 64, 512)), (4, (64, 96, 160), 8, (64, 32, 256)), (8, (128, 48, 80), 16, (128, 16, 128)), (16, (256, 24, 40), 32, (256, 8, 64)), (32, (512, 12, 20), 64, (512, 4, 32))], 'camera_05': [(2, (64, 192, 320), 4, (64, 64, 512)), (4,

In [10]:
model = PanoSupervisedModel(config.arch)

In [11]:
model.modules

<bound method Module.modules of PanoSupervisedModel(
  (blocks): ModuleDict()
  (networks): ModuleDict()
  (losses): ModuleDict()
  (flow_reverse): FlowReversal()
)>

In [12]:
model

PanoSupervisedModel(
  (blocks): ModuleDict()
  (networks): ModuleDict()
  (losses): ModuleDict()
  (flow_reverse): FlowReversal()
)

In [13]:
type(model)

vidar.arch.models.depth.PanoSupervisedModel.PanoSupervisedModel

In [14]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_forward_unimplemented',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_input_keys',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_n

In [15]:
config.arch.networks

Config(depth=Config(decoder=Config(activation='sigmoid', depth_focal=1.0, depth_hypothesis=[2, 3, 10, 30, 60, 90], downsample=False, file='decoders/PanoDepthDecoder', fusion_type='MultiDepthSweepFusion', input_cameras=['camera_01', 'camera_05', 'camera_06', 'camera_07', 'camera_08', 'camera_09'], num_ch_out=1, out_shape=[128, 1024], positional_encoding=16, ref_shape=[256, 2048], scale_and_shapes={'camera_01': [(2, (64, 192, 320), 4, (64, 64, 512)), (4, (64, 96, 160), 8, (64, 32, 256)), (8, (128, 48, 80), 16, (128, 16, 128)), (16, (256, 24, 40), 32, (256, 8, 64)), (32, (512, 12, 20), 64, (512, 4, 32))], 'camera_05': [(2, (64, 192, 320), 4, (64, 64, 512)), (4, (64, 96, 160), 8, (64, 32, 256)), (8, (128, 48, 80), 16, (128, 16, 128)), (16, (256, 24, 40), 32, (256, 8, 64)), (32, (512, 12, 20), 64, (512, 4, 32))], 'camera_06': [(2, (64, 192, 320), 4, (64, 64, 512)), (4, (64, 96, 160), 8, (64, 32, 256)), (8, (128, 48, 80), 16, (128, 16, 128)), (16, (256, 24, 40), 32, (256, 8, 64)), (32, (512,

In [16]:
model.modules

<bound method Module.modules of PanoSupervisedModel(
  (blocks): ModuleDict()
  (networks): ModuleDict()
  (losses): ModuleDict()
  (flow_reverse): FlowReversal()
)>

In [17]:
from vidar.core.wrapper import Wrapper
from vidar.utils.setup import setup_arch
from vidar.utils.networks import load_checkpoint

In [19]:
wrapper = Wrapper(config)

AssertionError: Path /data/datasets/DDAD/ddad_train_val/000000 does not exist